In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime

In [2]:
now = datetime.now()
options = Options() 
options.add_experimental_option("excludeSwitches", ['enable-automation']) # allows us to scrape bloomberg
#options.add_argument("--disable-notifications") #just added
#options.add_argument("--start-maximized")
tickers = ['ADOM','BKCH','AESEW','AMR','AMRWW','BLU','BIQI','CLDC','CNTF','CCIH','CRAY','DFRG','DVCR','EMMA','EMMAW','FNSR','FRED','GLADN','HUNT','HUNTU','HUNTW','IVENC','IVFGC','IVFVC','SKIS','REED','TBRGU','SFLY','SLNG','USAT','USATP','BCACU','XYN','XYNPW','YRIV']

In [4]:
# gathers list of bloomberg articles for a keyword
# bloomberg search replacement
def fetch_links(link):
    browser = webdriver.Chrome(chrome_options=options, executable_path=r"/usr/bin/chromedriver")
    browser.get(link)
    time.sleep(3)
    actions = ActionChains(browser)
    
    browser.find_element_by_css_selector("div.container__0c254cfc > div.container__9dcb6ba7:nth-child(2) > div.listWrap__6cc206d2 > a:nth-child(2)").click()
    time.sleep(1)
    max = 5
    counter = 0
    while browser.find_elements_by_css_selector("section.mainContent__b8b5cf5e > div:last-child > div:last-child > button"):
        if counter == 5:
            break
        for _ in range(3):   #Adjust this range according to your need, I meant how far you wanna go down.
            actions.send_keys(Keys.SPACE).perform()
            time.sleep(1)
        browser.find_element_by_css_selector("section.mainContent__b8b5cf5e > div:last-child > div:last-child > button").click()
        time.sleep(3)
        counter += 1
        
    links = scrapeSearch(browser.page_source)
    browser.quit()
    
    return links

def get_ticker_links(ticker):
    tick = fetch_links('https://www.bloomberg.com/search?query=%s'%(ticker))
    tick_raw = []
    i = 0
    while i < len(tick):
        articleSource = tick[i][1]['href']
        year = tick[i][0].text
        tick_raw.append([ticker, year, articleSource])
        i+=1
    tick_df = pd.DataFrame(tick_raw, columns=['ticker', 'year', 'url'])
    return tick_df
    #df = df.append(tick_df, ignore_index=True)

'''
def bloombergSearch(url):
    #url = 'https://www.bloomberg.com/search?query=%s'%(ticker)
    browser = webdriver.Chrome(chrome_options=options, executable_path=r"/usr/bin/chromedriver") # path should be similar for ubuntu vm
    browser.get(url)
    time.sleep(5) # wait for the page to load javascript
    pageSource = browser.page_source
    browser.quit()
    return pageSource
'''

def bloombergArticle(url):
    browser = webdriver.Chrome(chrome_options=options, executable_path=r"/usr/bin/chromedriver")
    browser.delete_all_cookies() # bypass bloomberg article limit
    browser.get(url)
    time.sleep(5) # wait for the page to load javascript
    pageSource = browser.page_source
    browser.quit()
    return pageSource

def contentCat(content):
# concatenate content strings of article
    article = ''
    for par in content:
        if par == content[-1]:
            article += par.get_text()
        else:
            article += par.get_text() + '\n'
    return article

def scrapeSearch(pageSource):
    soup = BeautifulSoup(pageSource, 'html.parser')
    searchResults = soup.select('div>div>section:nth-child(3)>div:last-child>div>a', id='root')
    year = soup.select('div>div>section:nth-child(3)>div:last-child>div>div>div.publishedAt__79f8aaad', id='root')
    return list(zip(year,searchResults))

def scrapeArticle(pageSource):
    soup = BeautifulSoup(pageSource, 'html.parser')
    url = soup.select('head>title+link')[0]['href']
    if soup.find_all('h1'):
        title = soup.find_all('h1')[0].get_text()
    elif soup.find_all('div.lede-newsletter > div:nth-child(2)'):
        title = soup.find_all('div.lede-newsletter > div:nth-child(2)')[0].get_text()
    else: 
        title = np.nan
    if soup.find_all(class_='author-v2__byline'):    
        author = soup.find_all(class_='author-v2__byline')[0].get_text()
    else:
        author = np.nan
    posted = soup.find_all('time', class_='article-timestamp')[0].get_text() # time of publish
    '''
    if len(soup.find_all('time', class_='article-timestamp')) == 2: 
        edited = soup.find_all('time', class_='article-timestamp')[1].get_text() # if no update date is available, uses publish date
    else: 
        edited = np.nan
    '''
    scraped = now.strftime("%d/%m/%Y %H:%M:%S")
    content = soup.select('div>p', class_='body-copy-v2')
    article = contentCat(content)
    return [url, title, author, posted, scraped, article]

In [5]:
ADOM_df = get_ticker_links('ADOM')

/home/albert/anaconda3/envs/cs/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


WebDriverException: Message: chrome not reachable
  (Session info: chrome=77.0.3865.90)


In [ ]:
df = pd.DataFrame(columns=['STOCK_ID', 'YEAR', 'URL'])

In [ ]:
df = df.append(ADOM_df, ignore_index=True)

In [ ]:
# transfrom date to get int year
df['year'] = df['year'].apply(lambda x: x[-4:])
df['year'] = df['year'].apply(lambda x: int(x))

In [ ]:
df_2019 = df[df['year'] == 2019]
bloomberg_df = pd.DataFrame(columns=['URL', 'HEADLINE', 'author', 'DATE_OF_PUBLISH', 'DATE_OF_SCRAPE', 'CONTENT'])

In [ ]:
STOCK_ID,URL,DATE_OF_PUBLISH,HEADLINE,CONTENT